<a href="https://colab.research.google.com/github/R-aryan/Image_Classification_VGG16/blob/master/Classification_Cat_VS_Dogs_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

import shutil
from os import listdir
from os.path import splitext

In [0]:
train_directory= "/content/drive/My Drive/classification_Dataset/cat_VS_dogs/train"
test_directory="/content/drive/My Drive/classification_Dataset/cat_VS_dogs/test1"

In [0]:
src= '/content/drive/My Drive/classification_Dataset/cat_VS_dogs/train'
dest_d='/content/drive/My Drive/classification_Dataset/cat_VS_dogs/train/Dogs'
dest_c='/content/drive/My Drive/classification_Dataset/cat_VS_dogs/train/Cats'
validation_set='/content/drive/My Drive/classification_Dataset/cat_VS_dogs/validation_data'